# Genetic Algorithms example: the knapsack problem

Hi! 

In this notebook we'll work out through one of the most basic NP-complete problems - the Knapsack Problem:

https://en.wikipedia.org/wiki/Knapsack_problem

Though it's decsription is quite basic, the algorithm to find it's best solution has a complexity of $O(2^n)$ - where $n$ is the number of possible items to put in the knapsack. 

This is _bad_.

We'll see how it can be solved with a genetic algorithm!

In [1]:
import numpy as np
import matplotlib.pyplot as plt

The knapsack problem is defined by:

* $N$ - the number of items
* $C$ - the capacity of knapsack
* $X$
* $$ -
*